In [180]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [205]:
data = pd.read_csv('data/round_train.csv')

In [206]:
data = data[data.Tournament_Year.isin([2003,2004])]

In [207]:
data.shape

(38626, 12)

In [208]:
data.head()

,Tournament_Year,Tournament_#,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Tee_Time,Round_Score,End_of_Event_Pos._(text),Finishing_Pos,Finishing_Pct
0,2003,10,16,656,1320,"Estes, Bob",1,11:50,66,T6,7.5,0.197917
1,2003,10,16,656,1361,"Forsman, Dan",1,11:00,69,T21,22.0,0.600694
2,2003,10,16,656,1761,"Mattiace, Len",1,13:00,75,24,24.0,0.656250
3,2003,10,16,656,1797,"Mediate, Rocco",1,12:10,72,T2,2.5,0.059028
4,2003,10,16,656,1920,"Parry, Craig",1,11:20,71,T25,26.5,0.725694


In [209]:
def label_players(player_inds,players):
    groups_d = {}
    for player in players:
        if sum(player not in player_ind for player_ind in player_inds)==5:
            probs = np.array([1/float(len(player_ind))**2 if len(player_ind)!=0 else 1 for player_ind in player_inds])
            #print p,p.sum()
            probs = probs/float(probs.sum(0))
            g = np.random.choice(range(5),1,p=probs)[0]
            player_inds[g][player] = len(player_inds[g])
            groups_d[player] = g
    indexes = []
    groups = []
    for player in players:
        indexes.append(player_inds[groups_d[player]][player])
        groups.append(groups_d[player])
    return (player_inds,indexes,groups)

In [210]:
player_inds = [{} for _ in range(5)]
player_inds,indexes,groups = label_players(player_inds,data.Player_Number.values)
player_inds_reverse = [dict((v, k) for k, v in player_ind.iteritems()) for player_ind in player_inds]

In [211]:
data.insert(5,'Group_Number',groups)
data.insert(6,'Index_Number',indexes)

In [212]:
rounds = data.groupby(['Tournament_Year','Tournament_#','Round_Number'])

In [213]:
len(rounds)

371

In [214]:
len(pd.unique(data.Player_Number))

917

In [222]:
data[data.Player_Name=='Woods, Tiger']
#data.head()

,Tournament_Year,Tournament_#,Permanent_Tournament_#,Course_#,Player_Number,Group_Number,Index_Number,Player_Name,Round_Number,Tee_Time,Round_Score,End_of_Event_Pos._(text),Finishing_Pos,Finishing_Pct
2226,2003,60,4,104,8793,3,58,"Woods, Tiger",1,10:00,70,1,1.0,0.005531
2378,2003,60,4,4,8793,3,58,"Woods, Tiger",2,08:57,66,1,1.0,0.005531
2496,2003,60,4,4,8793,3,58,"Woods, Tiger",3,09:50,68,1,1.0,0.005531
2570,2003,60,4,4,8793,3,58,"Woods, Tiger",4,10:35,68,1,1.0,0.005531
2682,2003,70,7,500,8793,3,58,"Woods, Tiger",1,12:37,72,T5,5.5,0.047454
2824,2003,70,7,500,8793,3,58,"Woods, Tiger",2,08:12,68,T5,5.5,0.047454
2936,2003,70,7,500,8793,3,58,"Woods, Tiger",3,09:20,73,T5,5.5,0.047454
3010,2003,70,7,500,8793,3,58,"Woods, Tiger",4,08:50,65,T5,5.5,0.047454
4435,2003,120,9,9,8793,3,58,"Woods, Tiger",1,08:00,70,1,1.0,0.006345
4553,2003,120,9,9,8793,3,58,"Woods, Tiger",2,12:30,65,1,1.0,0.006345


In [216]:
def fullprint(*args, **kwargs):
  from pprint import pprint
  opt = np.get_printoptions()
  np.set_printoptions(threshold='nan')
  pprint(*args, **kwargs)
  np.set_printoptions(**opt)

In [217]:
n_players = [len(pd.unique(data[data.Group_Number == n].Player_Number)) for n in range(5)]
n_rounds = len(rounds)

In [218]:
n_players

[183, 194, 177, 175, 188]

In [219]:
A = [np.zeros((n,n_rounds,n)) for n in n_players]
G = [np.zeros((n,n_rounds)) for n in n_players]
for u,df in enumerate(rounds):
    df = df[1]
#     if u>600: continue
    df = df.sort_values('Round_Score')
    for g in range(5):
        df_g = df[df.Group_Number==g]
        n = len(df_g)
        G[g][df_g.Index_Number.values,u] = 1
        for row_ind in xrange(n):
            p_id = df_g.iloc[row_ind].Index_Number
            arr = df_g.Round_Score.values.astype(float) / df_g.iloc[row_ind].Round_Score
            arr[row_ind] = 0
            A[g][p_id,u,df_g.Index_Number.values] += arr / (n-1)
#         fig, ax = plt.subplots(figsize=(20, 10))
#         plt.imshow(A[g][0,:,:])
#         plt.show()
        #fullprint(A[0,0:10,0:5])
        V[g] = np.ndarray((n_players[g],n_players[g]))
        for p in df_g.Index_Number:
            weights = np.matrix(G[g][p,:] / G[g].sum(1)[p])
            V[g][p,:] = (weights * A[g][p,:,:]).A1
        #fullprint(V[0:4,0:4])

In [220]:
for g in range(5):
    print 'Group %d' % (g+1,)
    p_nums = V[g].sum(1).argsort().tolist()[::-1][:20]
    for u,p in enumerate(p_nums):
        if G[g].sum(1)[p] > 6:
            print data[(data.Group_Number==g) & (data.Index_Number==p)].iloc[0].Player_Name,np.sort(V[g].sum(1)).tolist()[::-1][u],G[g].sum(1)[p]
    print '\n****************\n'

Group 1
Caron, Jason 1.88473048128 73.0
Herron, Tim 1.80900005991 183.0
Pappas, Brenden 1.80781392779 209.0
Day, Glen 1.80584553076 199.0
Sheehan, Patrick 1.77963597361 210.0
Huston, John 1.77693912819 129.0
Boros, Guy 1.62820560223 81.0
Villegas, Camilo 1.59016678485 30.0
Blackmar, Phil 1.47215671002 8.0

****************

Group 2
Singh, Vijay 1.03654646602 212.0
Els, Ernie 1.02896834619 118.0
Love III, Davis 1.02172298051 154.0
Mickelson, Phil 1.02011621668 157.0
Harrington, Padraig 1.01871570014 78.0
Verplank, Scott 1.01763292324 187.0
Johnson, Zach 1.01637916635 109.0
Haas, Jay 1.01619508891 162.0
Lehman, Tom 1.01399041347 148.0
Clarke, Darren 1.01394248431 99.0
Appleby, Stuart 1.01365490382 163.0
Couples, Fred 1.01222911888 119.0
Price, Nick 1.01185355241 112.0
Howell III, Charles 1.01137636604 216.0
Hensby, Mark 1.01085911104 96.0
Jacobson, Freddie 1.01080240569 109.0
Calcavecchia, Mark 1.00944554872 156.0

****************

Group 3
Clark, Tim 1.83096416159 162.0
Triplett, Kirk 1

In [223]:
V[3][58,:].sum()

1.0438055435875842